# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']
print('There are {} and {} entries in white and black datasets, respectively.'.format(len(w), len(b)))

There are 2435 and 2435 entries in white and black datasets, respectively.


### 1. What test is appropriate for this problem? Does CLT apply?

The question is to compare the probability of success, defined as interview requrest here, in two populations. A two sample z test would be appropriate. Given the number of samples are very large, the central limit theorom would apply.

### 2. What are the null and alternate hypotheses?

Null hypothesis: The probability of interview requrest in the black population is the same as that in the white population.
Alternate hypothesis: The probability of interview requrest in the black population is **not** the same as that in the white population.

### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [14]:
#Use the bootstrapping approach first
#Simulate 10000 bootstrap sampling of both datasets, record the difference between the two.
diff_bs = np.empty(10000)
for i in range(10000):
    w_sample = np.random.choice(w.call, len(w))
    b_sample = np.random.choice(b.call, len(b))
    diff_bs[i] = np.mean(w_sample) - np.mean(b_sample)
#Calculate 95% confidence interval
ci = np.percentile(diff_bs, [2.5, 97.5])
print ('The 95% confidence interval of the difference between white and black success probability\
 is between {:.3f} and {:.3f}.'.format(ci[0], ci[1]))
print ('The margin of error is {:.3f}.'.format((ci[1]-ci[0])/2))
#Calculate p-value, two-tailed
p = 2* np.sum(diff_bs<=0) / len(diff_bs)
print ('The p-value associated with the null hypothesis is {}.'.format(p))

The 95% confidence interval of the difference between white and black success probability is between 0.017 and 0.047.
The margin of error is 0.015.
The p-value associated with the null hypothesis is 0.0.


In [23]:
#Using frequentist statistical approach
#The variance of new variable is the sum of variance of each variable
ci = stats.norm.interval(0.95, np.mean(w.call)-np.mean(b.call), np.sqrt(np.var(w.call)/len(w) + np.var(b.call)/len(b)))
print ('The 95% confidence interval of the difference between white and black success probability\
 is between {:.3f} and {:.3f}.'.format(ci[0], ci[1]))
print ('The margin of error is {:.3f}.'.format((ci[1]-ci[0])/2))     
#Calculate the z score
z = (np.mean(b.call) - np.mean(w.call)) / np.sqrt(np.var(w.call)/len(w) + np.var(b.call)/len(b))
print('z = {}'.format(z))
#Look up the p-value, two-tailed, multiply by 2
p = 2 * stats.norm.cdf(z)
print('p = {}'.format(p))                      

The 95% confidence interval of the difference between white and black success probability is between 0.017 and 0.047.
The margin of error is 0.015.
z = -4.115583524916792
p = 3.8620110799840656e-05


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

### 4. Write a story describing the statistical significance in the context or the original problem.

In [25]:
alpha = 0.05
if p < alpha:
    print('The null hypothesis is rejected. There is statistically significant difference between the probablity of \
interview request of white-sounding applicants and that of black-sounding applicants.')
else:
    print('The null hypothesis cannot be rejected. There is not enough statistical evidence for difference between \
the probability of interview request of white-sounding applicants and that of black-sounding applicants.')

The null hypothesis is rejected. There is statistically significant difference between the probablity of interview request of white-sounding applicants and that of black-sounding applicants.


### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No. The impact of other factors on the callback success rate has not been evaluated. To answer this question, one would need to quantify the impact of each factor on callback success and compare them. One simple way to do this would be using the Pearson correlation coefficient.

In [26]:
#First, let's take a look at the data
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [30]:
#There are several categorical data. Let's change the race and sex column into numerical data.
#Others are harder to deal with. Let's leave them for now.
data['white'] = pd.get_dummies(data['race'], drop_first=True)
data['male'] = pd.get_dummies(data['sex'], drop_first=True)

In [41]:
#Calculate correlation matrix
data_corr = data.corr()
data_corr['call'].abs().sort_values(ascending=False).head(10)

call                1.000000
specialskills       0.111074
honors              0.071951
empholes            0.071888
adid                0.063178
yearsexp            0.061436
white               0.058872
fracdropout         0.056671
lmedhhinc_empzip    0.049879
linc                0.049649
Name: call, dtype: float64

Looks like special skills is the most important factor for callback success. Several other factors such as honors and years of experiences are all more important factor than race.